# Lexicons

Lexicon definitions live in `lexicons` directory. tinychat's base NSID is:

`chat.tinychat`

Lexicons specs get converted to TS using

`deno task generate:lexicons`

**Note:** I've also included `com.atproto` lexicons - otherwise generated API
code for tinychat references types that are not available (for things like
record CRUD ops)

Random notes:

```
   chat.tinychat.actor

   chat.tinychat.server <-- chat servers I created 
      - tinychat dev channel
      - trumpet players

 
   chat.tinychat.channel
      - general @ tinychat dev channel
      - random @ tinychat dev channel

   chat.tinychat.message (each user's message are on his/her data repo)
      - DID identifies sender

   chat.tinychat.membership
      (use strong refs to reference server: https://github.com/bluesky-social/atproto/blob/main/lexicons/app/bsky/feed/like.json#L13)
      - server
```

In [ ]:
import { merman } from "tinychat/tools/merman.ts";
import { lexicons } from "tinychat/api/lexicons.ts";
import { LexUserType } from "@atproto/lexicon";
import { schema } from "@atproto/common";
import {TID} from "@atproto/common";

const [name, def] = lexicons.defs.entries().toArray()[0];
const defAliases: Record<string, string> = {};

// generate random alpha string
const genAlias = (length: number = 10) => {
  const chars = "abcdefghijklmnopqrstuvwxyz";
  let result = "";
  for (let i = length; i > 0; --i) {
    result += chars[Math.floor(Math.random() * chars.length)];
  }
  return result;
};

const diagram = (name: string, def: LexUserType): string | undefined => {
  if (def.type === "object") {
    const props = Object.entries(def.properties).map((prop) => {
      const [name, d] = prop;

      if (d.type === "string") {
        return `${name} ${d.type} "${d.format} ${d.description}"`;
      }

      return `${name} ${d.type}`;
    });

    defAliases[name] = genAlias();

    return `
    ${defAliases[name]}["${name}"] {
      ${props.join("\n")}
     }`;
  }
  return "";
};

const d = `---
config:
  fontSize: 18
  theme: neutral
  layout: elk
  elk:
    mergeEdges: true
    nodePlacementStrategy: NETWORK_SIMPLEX
---
erDiagram
  ${lexicons.defs
    .entries()
    .toArray()
    .filter((e) => e[0].match(/tinychat/gi))
    .map((e) => diagram(e[0], e[1]))
    .filter((e) => e)
    .join("\n")}
`;
console.log(d);

merman(d,`<strong>Legend:</strong>hello<hr/>`);

// merman(`
// erDiagram
//     "chat.tinychat.actor.defs.actorView" {
//         did string "did"
//         handle string
//         displayName string "maxGraphemes: 64, maxLength: 2560"
//         description string "maxGraphemes: 64, maxLength: 2560"
//         avatar string "uri"
//     }
//     "chat.tinychat.core.message.record" {
//         text string
//         server string
//         channel string
//         reply replyRef
//     }
//     "chat.tinychat.core.message.replyRef" {
//         root ref "lex:com.atproto.repo.strongRef"
//         parent ref "lex:com.atproto.repo.strongRef"
//     }
// `);

---
config:
  fontSize: 18
  theme: neutral
  layout: elk
  elk:
    mergeEdges: true
    nodePlacementStrategy: NETWORK_SIMPLEX
---
erDiagram
  
    oobbfrnzdu["lex:chat.tinychat.actor.defs#actorView"] {
      did string "did undefined"
handle string "handle undefined"
displayName string "undefined undefined"
description string "undefined undefined"
avatar string "uri undefined"
     }

    awdajncygd["lex:chat.tinychat.core.message#replyRef"] {
      root ref
parent ref
     }

    kqkdxujgcp["lex:chat.tinychat.core.server#channelRef"] {
      id string "tid undefined"
name string "undefined Channel name"
     }

    jjzanyqkwr["lex:chat.tinychat.richtext.facet#main"] {
      index ref
features array
     }

    hlapcafnqy["lex:chat.tinychat.richtext.facet"] {
      index ref
features array
     }

    revzfraoce["lex:chat.tinychat.richtext.facet#mention"] {
      did string "did undefined"
     }

    gjwahuowzg["lex:chat.tinychat.richtext.facet#link"] {
      uri string "uri undefi

Listening on http://0.0.0.0:7864/


## Using lexicons

Validate record

In [ ]:
import { assert } from "asserts";
import { validateRecord } from "tinychat/api/types/chat/tinychat/core/server.ts";
import { TID } from "@atproto/common";

Deno.test("validate records using lexicons", () => {
  // names must be a string + missing channels
  let r = validateRecord({
    name: 5,
  });
  // @ts-ignore error is not defined
  assert(!r.success, r.error);

  // is required
  r = validateRecord({});
  // @ts-ignore error is not defined
  assert(!r.success, r.error);

  // all good
  r = validateRecord({
    name: "test",
    channels: [
      {
        name: "test",
        id: TID.nextStr(),
      },
    ],
  });
  // @ts-ignore error is not defined
  assert(r.success, r.error);
});

Let's test CRUD

In [ ]:
import { TinychatAgent } from "tinychat/agent.ts";
import { TID } from "@atproto/common";
import { assert, assertRejects } from "asserts";
import { validateRecord } from "tinychat/api/types/chat/tinychat/core/server.ts";

Deno.test("test CRUD", async () => {
  const ta = await TinychatAgent.create();
  const name = `test-${TID.nextStr()}`;

  const r = await ta.chat.tinychat.core.server.create(
    {
      repo: ta.agent.assertDid,
    },
    {
      name,
      channels: [
        {
          name: "test",
          id: TID.nextStr(),
        },
      ],
    },
  );

  const rkey = r.uri.split("/").pop() || "";

  const rec = await ta.chat.tinychat.core.server.get({
    repo: ta.agent.assertDid,
    rkey,
  });

  assert(rec);
  assert(rec.value.name === name);
  assert(rec.uri === r.uri);

  const { records } = await ta.chat.tinychat.core.server.list({
    repo: ta.agent.assertDid,
    limit: 10,
  });

  assert(records.length <= 10);
  assert(records.find((r) => r.uri === rec.uri));

  await ta.chat.tinychat.core.server.delete({
    repo: ta.agent.assertDid,
    rkey,
  });

  assert(
    !(
      await ta.chat.tinychat.core.server.list({
        repo: ta.agent.assertDid,
        limit: 10,
      })
    ).records.find((r) => r.uri === rec.uri),
  );
});

Deno.test("test validation", async () => {
  const ta = await TinychatAgent.create();

  await assertRejects(() =>
    ta.chat.tinychat.core.server.create(
      { repo: ta.agent.assertDid },
      // @ts-ignore yolo
      validateRecord({ name: 5 }).value,
    )
  );
});

Deno.test("test basic data model", async () => {
  const ta = await TinychatAgent.create();
  const repo = ta.agent.assertDid;
  const channelId = TID.nextStr();

  // create a server
  const { uri } = await ta.chat.tinychat.core.server.create(
    { repo },
    {
      name: "tinychat dev community",
      channels: [
        {
          name: "test",
          id: channelId,
        },
      ],
    },
  );

  // join it

  await ta.chat.tinychat.core.membership.create({ repo }, {
    server: uri,
    createdAt: new Date().toISOString(),
  });

  // add channels to the server

  // message the channel
  const message = await ta.chat.tinychat.core.message.create(
    { repo },
    {
      server: uri,
      channel: channelId,
      text: "hello world",
      createdAt: new Date().toISOString(),
    },
  );

  // reply to it
  await ta.chat.tinychat.core.message.create(
    { repo },
    {
      channel: channelId,
      text: "nice post",
      server: uri,
      createdAt: new Date().toISOString(),
      reply: {
        root: { uri: message.uri, cid: message.cid },
        parent: { uri: message.uri, cid: message.cid },
      },
    },
  );
});